In [1]:
!pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 3.9 MB/s eta 0:00:0000:0100:01


In [6]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 KB 1.1 MB/s eta 0:00:00


In [9]:
import pymysql
from datetime import datetime


In [10]:
PWA_DB_HOST_V12CHAT_READ="preproduction-db-1-instance-1.c7nkisio0atn.us-west-2.rds.amazonaws.com"
PWA_DB_PORT_V12CHAT_READ="3306"
PWA_DB_DATABASE_V12CHAT_READ="v12_conversation"
PWA_DB_USERNAME_V12CHAT_READ="v12_conversation"
PWA_DB_PASSWORD_V12CHAT_READ="V12_c0nvers&tion_2023"
#############################################
PWA_DB_HOST_V12CHAT_WRITE="preproduction-db-1-instance-1.c7nkisio0atn.us-west-2.rds.amazonaws.com"
PWA_DB_PORT_V12CHAT_WRITE="3306"
PWA_DB_DATABASE_V12CHAT_WRITE="v12_conversation"
PWA_DB_USERNAME_V12CHAT_WRITE="v12_conversation"
PWA_DB_PASSWORD_V12CHAT_WRITE="V12_c0nvers&tion_2023"
#############################################
PWA_DB_HOST_V12_ACCOUNT_READ="preproduction-db-1-instance-1.c7nkisio0atn.us-west-2.rds.amazonaws.com"
PWA_DB_PORT_V12_ACCOUNT_READ="3306"
PWA_DB_DATABASE_V12_ACCOUNT_READ="v12_account"
PWA_DB_USERNAME_V12_ACCOUNT_READ="v12_account"
PWA_DB_PASSWORD_V12_ACCOUNT_READ="PHR6aHLcHx!aM6Wk"

In [13]:
import pymysql
from datetime import datetime

class DataBase:
    # Connection to any database using HOST, USER, PASSWORD, NAME
    def connexion(self, host, user, password, database):
        try:
            conn = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )
            return conn
        except pymysql.Error as e:
            error_mes = 'Connexion function => ' + str(e)
            now = datetime.now()
            text = f'{{"@timestamp":"{now}","log.level":"ERROR","app_name":"Bot","Channel":"PWA","message":"{error_mes}"}}'
            print(text)  # Consider logging it to a file instead
            return None

    # Get any value from database using SELECT query
    def readQuery(self, conn, query, data=None):
        try:
            with conn.cursor() as cur:
                if data is not None:
                    cur.execute(query, data)
                else:
                    cur.execute(query)
                rows = cur.fetchall()
            return rows
        except pymysql.Error as e:
            error_mes = 'readQuery function => ' + str(e)
            now = datetime.now()
            text = f'{{"@timestamp":"{now}","log.level":"ERROR","app_name":"Bot","Channel":"PWA","message":"{error_mes}"}}'
            print(text)
            return None

    # Insert any value in database using INSERT query
    def writeQuery(self, conn, query, data):
        try:
            with conn.cursor() as cursor:
                resp = cursor.execute(query, data)
                conn.commit()
            return resp
        except pymysql.Error as e:
            error_mes = 'writeQuery function => ' + str(e)
            now = datetime.now()
            text = f'{{"@timestamp":"{now}","log.level":"ERROR","app_name":"Bot","Channel":"PWA","message":"{error_mes}"}}'
            print(text)
            return 0

    # Update any table in database with UPDATE query
    def updateQuery(self, conn, query, data):
        return self.writeQuery(conn, query, data)

    # Delete rows from table
    def deleteQuery(self, conn, query, data):
        return self.writeQuery(conn, query, data)

    # Write to tracking conversation
    def writeTracking(self, conn, data):        
        query = """
        INSERT INTO tracking_conversation 
        (intent, step, understand, date_time, proposed_time, list_time, year, make, miles, vehicle_id, vehicle_name, finished, take_by_human, lead_id, dealer_id, date_without, last_response) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        return self.writeQuery(conn, query, data)

    # Write to tracking conversation SMS
    def writeTrackingSmsBdc(self, conn, data):        
        query = """
        INSERT INTO tracking_sms_conversations 
        (intent, step, date_time, proposed_time, list_time, vehicle_id, vehicle_name, lead_id, dealer_id, last_message, last_message_date, end_conversation, date_without) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        return self.writeQuery(conn, query, data)

In [15]:
db = DataBase()

# Example function using the instance
def _shipping(DB, dealer_id):
    shipping = None
    shippingQuery = "SELECT is_shipping FROM chat_settings WHERE user_id = %s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    if conn_DBC:
        result = DB.readQuery(conn_DBC, shippingQuery, (dealer_id,))
        if result:
            shipping = result[0][0]
        conn_DBC.close()  # Ensure connection is closed
    return shipping

In [19]:
def dealership_name(DB, dealer_id):
    dealer_name = None
    try:
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        queryPROD = "SELECT company_name FROM dealer_address WHERE user_id = %s"
        res = DB.readQuery(conn_Prod, queryPROD, dealer_id)
        dealer_name = res[0][0]
    except:
        pass
    return dealer_name

In [20]:
print(_shipping(DB=db,dealer_id=40255))

0


In [21]:
print(dealership_name(DB=db,dealer_id=40255))

add-dev3


In [14]:
def _shipping(DB, dealer_id):
    shipping=None
    shippingQuery = "SELECT is_shipping FROM `chat_settings` where user_id=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    shippingRes = DB.readQuery(conn_DBC, shippingQuery, dealer_id)
    if shippingRes[0][0]==0:
        shipping= "not offer shipping"
    else:
        shipping= "we offer shipping"
    return shipping
def dealer_time_zone(DB, dealer_id):
    query_time = "SELECT time_zone FROM `dealer_address` WHERE user_id= %s"
    conn_Prod = DB.connexion(
        PWA_DB_HOST_V12_ACCOUNT_READ,
        PWA_DB_USERNAME_V12_ACCOUNT_READ,
        PWA_DB_PASSWORD_V12_ACCOUNT_READ,
        PWA_DB_DATABASE_V12_ACCOUNT_READ,
    )
    res_type = DB.readQuery(conn_Prod, query_time, dealer_id)
    timeZone = res_type[0][0]
    return timeZone
# function to get dealership name
def dealership_name(DB, dealer_id):
    dealer_name = None
    try:
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        queryPROD = "SELECT company_name FROM dealer_address WHERE user_id = %s"
        res = DB.readQuery(conn_Prod, queryPROD, dealer_id)
        dealer_name = res[0][0]
    except:
        pass
    return dealer_name
# get address and link of location for dealer
def address_location(dealer_id, DB):
    conn_Prod = DB.connexion(
        PWA_DB_HOST_V12_ACCOUNT_READ,
        PWA_DB_USERNAME_V12_ACCOUNT_READ,
        PWA_DB_PASSWORD_V12_ACCOUNT_READ,
        PWA_DB_DATABASE_V12_ACCOUNT_READ,
    )
    queryPROD = (
        "SELECT  address, city, state_id, zip FROM dealer_address WHERE user_id = %s"
    )
    res = DB.readQuery(conn_Prod, queryPROD, dealer_id)
    address = res[0][0]
    city = res[0][1]
    state_id = res[0][2]
    zipcode = res[0][3]
    querystate = "SELECT  state_name FROM states WHERE id = %s"
    state_res = DB.readQuery(conn_Prod, querystate, state_id)
    state = state_res[0][0]
    addcty = address + ",+" + city
    new_adress = addcty.replace(" ","+")
    dealer_address = address + " " + str(city) + " " + str(state) + " " + str(zipcode)
    link = "https://www.google.com/maps/search/?api=1&query=" + new_adress
    return [dealer_address, link]
# function to get dealer phone
def get_dealer_phone(DB, dealer_id):
    query1 = "SELECT `display_phone_number` FROM `chat_settings` WHERE `user_id`= %s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    res_link = DB.readQuery(conn_DBC, query1, dealer_id)
    if res_link[0][0] is not None and res_link[0][0] != "" and res_link[0][0] != "null":
        phone = res_link[0][0]
    else:
        query_phone = "SELECT cell_phone FROM `dealer_address` WHERE user_id=%s"
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        phone_res = DB.readQuery(conn_Prod, query_phone, dealer_id)
        phone = phone_res[0][0]
    phone = checkPhone(phone)
    phone = "(" + phone[:3] + ") " + phone[3:6] + "-" + phone[6:]
    return phone
#function to get inventory url
def inventory_url(DB, dealer_id):
    query1 = "SELECT `is_inventory_internal_link`,`external_inventory_link` FROM `chat_settings` WHERE `user_id`= %s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    res_link = DB.readQuery(conn_DBC, query1, dealer_id)
    if res_link[0][0] == 0:
        link = res_link[0][1]
        if link is not None:
            return link
    query2 = "SELECT name FROM domain_name WHERE user_id = %s"
    conn_Prod = DB.connexion(
        PWA_DB_HOST_V12_ACCOUNT_READ,
        PWA_DB_USERNAME_V12_ACCOUNT_READ,
        PWA_DB_PASSWORD_V12_ACCOUNT_READ,
        PWA_DB_DATABASE_V12_ACCOUNT_READ,
    )
    res = DB.readQuery(conn_Prod, query2, dealer_id)
    inventory_link = res[0][0] + "/all-inventory"
    return inventory_link
# function to get the credit application link for dealer
def financing_url(DB, dealer_id):
    financing_link = None
    financeQuery = "SELECT is_financing, external_financing_link, is_internal_link, type_financing_id, credit_score, other_financing FROM `chat_settings` where user_id=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    financingRes = DB.readQuery(conn_DBC, financeQuery, dealer_id)
    is_financing = financingRes[0][0]
    external_financing_link = financingRes[0][1]
    is_internal_link = financingRes[0][2]
    finance_type=financingRes[0][3]
    credit_score=financingRes[0][4]
    other_financing=financingRes[0][5]
    if is_financing == 0 or is_financing == '0' :
        return ["This dealer not offer financing", finance_type,credit_score, other_financing ]
    if is_internal_link == 0 or is_internal_link == '0':
        financing_link = external_financing_link
    else:
        query2 = "SELECT name FROM domain_name WHERE user_id = %s"
        conn_Prod = DB.connexion(
            PWA_DB_HOST_V12_ACCOUNT_READ,
            PWA_DB_USERNAME_V12_ACCOUNT_READ,
            PWA_DB_PASSWORD_V12_ACCOUNT_READ,
            PWA_DB_DATABASE_V12_ACCOUNT_READ,
        )
        res = DB.readQuery(conn_Prod, query2, dealer_id)
        financing_link = res[0][0] + "/credit-application"
    return [financing_link, finance_type,credit_score, other_financing ]
# function to get welcome message for facebook MarcketPlace
def get_welcome_message(DB, dealer_id):
    query = "SELECT is_custom_fbmp_message , content_custom_fbmp_message FROM `chat_settings` WHERE `user_id`=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    welcome_msg = DB.readQuery(conn_DBC, query, dealer_id)
    if welcome_msg[0][0] == 1 or welcome_msg[0][0] == "1":
        return welcome_msg[0][1]
    else:
        return None
#function to get custmize message when informations of car not provide
def message_no_informations(DB, dealer_id):
    query = "SELECT is_custom_no_information , content_custom_no_information	 FROM `chat_settings` WHERE `user_id`=%s"
    conn_DBC = DB.connexion(
        PWA_DB_HOST_V12CHAT_READ,
        PWA_DB_USERNAME_V12CHAT_READ,
        PWA_DB_PASSWORD_V12CHAT_READ,
        PWA_DB_DATABASE_V12CHAT_READ,
    )
    no_infos_msg = DB.readQuery(conn_DBC, query, dealer_id)
    if no_infos_msg[0][0] == 1 or no_infos_msg[0][0] == "1":
        return no_infos_msg[0][1]
    else:
        return None  
#Api to get opening Hours
opening_hours_url = (
        f"https://websites.addstaging.com/api/external_api/hours?user_id={dealer_id}"
    )
    headers=create_token()
    r = requests.get(opening_hours_url, headers=headers)
#get Carfax link from Api
info_veh_url = f"https://inventory.addstaging.com/api/get-carfax-link?user_id={dealer_id}&vin={VIN}"
headers=create_token()
r = requests.get(info_veh_url,headers=headers)

IndentationError: unexpected indent (2515199596.py, line 180)